In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Tokenizer
from eunjeon import Mecab
from konlpy.tag import Okt
from tokenizers import BertWordPieceTokenizer

#Modeling
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
submission = pd.read_csv('sample_submission.csv')
topic_dict = pd.read_csv('topic_dict.csv')

In [3]:
def identity_tokenizer(text):
        return text

In [12]:
tokenizer = Mecab()
train_text = [tokenizer.morphs(t) for t in train['title']]
test_text = [tokenizer.morphs(t) for t in test['title']]

vect = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False) # 크기 20000으로 제한해도 성능 개선 X
vect = vect.fit(train_text) # train + test를 fit시켰을 때보다 train만 fit시켰을 때의 성능이 더 좋았음 (lr, lgbm)
matrix_train = vect.transform(train_text)
matrix_test = vect.transform(test_text)
print('Vocab size:', matrix_train.shape[1])

Vocab size: 30459


## LR

### ver1

### ver2

In [14]:
from sklearn.model_selection import GridSearchCV

grid={"C":np.logspace(-3,3,10), "penalty":["l1","l2"]}# l1 lasso l2 ridge
lr=LogisticRegression()
lr_cv = GridSearchCV(lr,grid,cv=10)
lr_cv.fit(matrix_train,train['topic_idx'])

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'penalty': ['l1', 'l2']})

In [15]:
lr_cv.best_params_

{'C': 2.154434690031882, 'penalty': 'l2'}

In [18]:
lr=LogisticRegression(C=2.154434690031882, penalty='l2') # random_state = 42 안함
lr.fit(matrix_train,train['topic_idx'])
pred = lr.predict(matrix_test)

## SVM

### ver1

In [30]:
svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [35]:
from sklearn.svm import SVC

param_grid ={"C" : [0.1, 1, 10, 100], "gamma" : ['scale', 'auto'], "kernels" : ['polynomial', 'rbf', 'sigmoid', 'linear', 'precomputed']}

svm_cv = GridSearchCV(SVC(), param_grid, cv=10)
svm_cv.fit(matrix_train,train['topic_idx'])
svm_cv.best_params_

ValueError: Invalid parameter kernels for estimator SVC(C=0.1). Check the list of available parameters with `estimator.get_params().keys()`.

In [34]:
svm = SVC(random_state = 42)
svm.fit(matrix_train,train['topic_idx'])
pred = svm.predict(matrix_test)